In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib.parse
import requests
import pandas as pd
import warnings
import re
import time

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


warnings.filterwarnings("ignore")

"""
This script scrapes job postings from LinkedIn for specific job titles and locations.

It uses Selenium to automate browser actions and BeautifulSoup to parse HTML content.
The extracted data is then saved to a CSV file.

Requirements:
    - Python 3.6+
    - Selenium
    - BeautifulSoup4
    - Pandas
    - Requests
    - webdriver-manager (install using: pip install webdriver-manager)
"""

# Define the login credentials
username = "kathisnehithreddy@gmail.com"
password = "Ksnehith@123"

# Initialize the Chrome driver
service = Service(executable_path=r"C:\Users\snehi\Desktop\Linkedin_extract\chromedriver.exe")
driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome(executable_path=r"C:\Users\snehi\Desktop\Linkedin_extract\chromedriver.exe") # latest 130 chrome driver
driver.maximize_window()

# Navigate to the LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Find the input fields and enter the login credentials
username_field = driver.find_element(By.ID, "username")
username_field.send_keys(username)

password_field = driver.find_element(By.ID, "password")
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.RETURN)

# Wait for the page to load after login
time.sleep(5)

# Set user agent for requests
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
headers = {'User-Agent': user_agent}

print("LinkedIn LOGGED IN Successfully......../Extracting the postings")

# Initialize data storage
data = []
data_col = ['Job_title', 'City', 'State', 'Country', 'Company', 'Post_date', 'Job_link', 'Company_link']

# Define job titles and locations to search for
job_titles = ['Data Analyst', 'Analyst', 'Business Analyst', "Medicaid analyst" ]  # List of job titles to search for
locations = ['Texas', 'California']  # List of locations to search in

# Main loop for scraping job postings
# Iterates through job titles, locations, and pages of search results
# Extracts relevant information from each job posting and appends to data list

for job_title in job_titles:
    for location in locations:
        for page in range(1):  # Iterate over multiple pages
            getVars = {'keywords': job_title, 'location': location, 'sort': 'date', 'start': str(page * 10), 'f_E': '1,2,3', 'f_TPR': 'r2592000'}
            url = 'https://www.linkedin.com/jobs/search/?' + urllib.parse.urlencode(getVars)
            print('First (in loop):', url)

            response = requests.get(url, headers=headers, verify=False)
            soup = BeautifulSoup(response.content, 'html.parser')
            soup
            extract_posting = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')

            for post in extract_posting:
                position_title = str(post.find('h3', class_='base-search-card__title').text.strip())
                
                location_full = str(post.find('span', class_='job-search-card__location').text.strip())
                location_parts = location_full.split(', ')
                
                city = location_parts[0] if len(location_parts) > 0 else None
                state = location_parts[1] if len(location_parts) > 1 else None
                country = location_parts[2] if len(location_parts) > 2 else None
                
                position_company = str(post.find('h4', class_="base-search-card__subtitle").text.strip())
                position_date = datetime.strptime(post.find('time')['datetime'], '%Y-%m-%d').date()
                position_Link = post.find('a', class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')['href']
                position_companylink = post.find('a', class_='hidden-nested-link')['href']

                data.append([
                    position_title, city, state, country, 
                    position_company, position_date, 
                    position_Link, position_companylink
                ])

# Create a Pandas DataFrame from the extracted data
# Create DataFrame
df = pd.DataFrame(data, columns=data_col)

# Save DataFrame to CSV
df.to_csv("linkedinJobs_data.csv", index=False)

print("CSV file saved successfully.")

LinkedIn LOGGED IN Successfully......../Extracting the postings
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Data+Analyst&location=Texas&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Data+Analyst&location=California&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Analyst&location=Texas&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Analyst&location=California&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Business+Analyst&location=Texas&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Business+Analyst&location=California&sort=date&start=0&f_E=1%2C2%2C3&f_TPR=r2592000
First (in loop): https://www.linkedin.com/jobs/search/?keywords=Medicaid+analyst&location